In [1]:
import os
import json

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score

# Quantum imports (Qiskit 1.x + qiskit-machine-learning)
from qiskit.circuit.library import ZZFeatureMap, TwoLocal
from qiskit.primitives import Estimator
from qiskit_algorithms.optimizers import COBYLA

from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier

# Paths
DATA_DIR = os.path.join("..", "data", "plasticc")
FEATURES_PATH = os.path.join(DATA_DIR, "transient_features.csv")

RESULTS_DIR = os.path.join("..", "results")
os.makedirs(RESULTS_DIR, exist_ok=True)

FEATURES_PATH, RESULTS_DIR

('../data/plasticc/transient_features.csv', '../results')

In [2]:
features_df = pd.read_csv(FEATURES_PATH)
features_df.head()

,transient_id,n_points,flux_min,flux_max,delta_flux,flux_mean,flux_std,time_span,max_slope,min_slope,mean_slope,label
0,73799799,146,-71.079659,174.403397,245.483056,6.333829,25.600407,1088.9187,396.000641,-79.409779,3.106681,SNIa
1,215282,352,-21.821419,26.039886,47.861305,1.982695,5.601848,873.7903,2266.541688,-2429.961380,-23.766219,SNIa
2,92999561,128,-56.247101,220.577240,276.824341,5.884043,35.070857,913.7473,1563.930964,-58.885288,14.755975,SNIa
3,19866,351,-11.846063,270.410736,282.256799,20.666038,49.216990,853.7060,13188.285782,-13483.200137,-39.150423,SNIa
4,68637164,106,-122.754425,119.619064,242.373489,3.275453,24.372910,912.7714,407.938275,-1304.584846,-12.743792,SNIa


In [3]:
# Use the engineered features (update if you renamed things)
selected_features = ["delta_flux", "flux_std", "time_span"]

for f in selected_features:
    if f not in features_df.columns:
        raise ValueError(f"Feature {f} not found in dataframe columns: {features_df.columns.tolist()}")

X = features_df[selected_features].values
y_str = features_df["label"].values

label_mapping = {"SNII": 0, "SNIa": 1}
y = np.array([label_mapping[val] for val in y_str])

X.shape, y.shape, np.unique(y, return_counts=True)

((600, 3), (600,), (array([0, 1]), array([300, 300])))

In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y,
)

X_train.shape, X_test.shape

((480, 3), (120, 3))

In [5]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Map to angles in [-π/2, π/2]
X_train_angles = (np.pi / 2.0) * X_train_scaled
X_test_angles = (np.pi / 2.0) * X_test_scaled

X_train_angles[:3]

array([[-0.21004759, -0.17150886, -0.59250258],
       [-0.25062977, -0.2312758 , -1.41832599],
       [ 0.09977606,  0.16031387, -0.8012525 ]])

In [6]:
num_features = X_train_angles.shape[1]
num_qubits = num_features

num_qubits

3

In [11]:
# Feature map: encodes classical features into rotations + entanglement
feature_map = ZZFeatureMap(
    feature_dimension=num_features,
    reps=1,          # you can try 2 later
    entanglement="linear"
)

# Ansatz: trainable part of the circuit
ansatz = TwoLocal(
    num_qubits=num_qubits,
    reps=2,          # circuit depth – can tune later
    rotation_blocks=["ry", "rz"],
    entanglement_blocks="cz",
    entanglement="linear"
)

# Full variational circuit = feature_map -> ansatz
from qiskit import QuantumCircuit

circuit = QuantumCircuit(num_qubits)
circuit.compose(feature_map, inplace=True)
circuit.compose(ansatz, inplace=True)

# Use text drawer to avoid extra dependencies


In [12]:
estimator = Estimator()

# Input parameters = classical feature angles
input_params = list(feature_map.parameters)

# Trainable parameters = ansatz weights
weight_params = list(ansatz.parameters)

len(input_params), len(weight_params)

/var/folders/4j/986s_f794zq87n31mh_7ls200000gn/T/ipykernel_80587/2743393232.py:1: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()


(3, 18)

In [13]:
qnn = EstimatorQNN(
    estimator=estimator,
    circuit=circuit,
    input_params=input_params,
    weight_params=weight_params,
)

qnn

/var/folders/4j/986s_f794zq87n31mh_7ls200000gn/T/ipykernel_80587/974032110.py:1: DeprecationWarning: V1 Primitives are deprecated as of qiskit-machine-learning 0.8.0 and will be removed no sooner than 4 months after the release date. Use V2 primitives for continued compatibility and support.
  qnn = EstimatorQNN(


In [14]:
optimizer = COBYLA(maxiter=80)  # you can increase later if you want

q_clf = NeuralNetworkClassifier(
    neural_network=qnn,
    optimizer=optimizer,
    one_hot=False,      # binary labels 0/1
    warm_start=False,
)

q_clf

In [15]:
%%time

q_clf.fit(X_train_angles, y_train)

CPU times: user 54.4 s, sys: 1.31 s, total: 55.7 s
Wall time: 1min 1s


In [21]:
from sklearn.metrics import roc_auc_score

# ---- 1. Get raw quantum predictions ----
y_pred_raw = q_clf.predict(X_test_angles)
print("Raw quantum preds (unique):", np.unique(y_pred_raw))

# Map raw labels to {0,1}
# Assume <= 0 -> class 0 (SNII), > 0 -> class 1 (SNIa)
y_pred_q = np.where(y_pred_raw <= 0, 0, 1)
print("Mapped preds (unique):", np.unique(y_pred_q))

# ---- 2. Get quantum "probabilities"/scores ----
y_proba_q = q_clf.predict_proba(X_test_angles)  # shape (n_samples, 1) in your case
print("y_proba_q shape:", y_proba_q.shape)

# Flatten to (n_samples,) – works for (n,1) or (n,)
scores = y_proba_q.ravel()
print("score min/max:", scores.min(), scores.max())

# ---- 3. Metrics ----
q_acc = accuracy_score(y_test, y_pred_q)
q_auc = roc_auc_score(y_test, scores)  # use 1D scores for AUC

print(f"\nQuantum (EstimatorQNN) accuracy: {q_acc:.3f}")
print(f"Quantum (EstimatorQNN) AUC:      {q_auc:.3f}\n")

print("Classification report (Quantum):")
print(
    classification_report(
        y_test,
        y_pred_q,
        labels=[0, 1],  # explicitly say which labels we care about
        target_names=["SNII (0)", "SNIa (1)"],
        zero_division=0,
    )
)

cm_q = confusion_matrix(y_test, y_pred_q, labels=[0, 1])
cm_q

Raw quantum preds (unique): [-1.  1.]
Mapped preds (unique): [0 1]
y_proba_q shape: (120, 1)
score min/max: -0.57778967024171 0.8096137754110566

Quantum (EstimatorQNN) accuracy: 0.475
Quantum (EstimatorQNN) AUC:      0.459

Classification report (Quantum):
              precision    recall  f1-score   support

    SNII (0)       0.20      0.02      0.03        60
    SNIa (1)       0.49      0.93      0.64        60

    accuracy                           0.47       120
   macro avg       0.34      0.48      0.34       120
weighted avg       0.34      0.47      0.34       120



array([[ 1, 59],
       [ 4, 56]])

In [23]:
# ============================================================================
# 03_quantum_classifier.ipynb - OPTION A: OUTLIER-ROBUST VERSION
# Quantum ML for PLAsTiCC Transient Classification
# ============================================================================

import os
import json
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.metrics import (
    accuracy_score, 
    classification_report, 
    confusion_matrix, 
    roc_auc_score
)

# Quantum imports (Qiskit 1.x + qiskit-machine-learning)
from qiskit.circuit.library import ZZFeatureMap, TwoLocal
from qiskit.primitives import Estimator
from qiskit_algorithms.optimizers import COBYLA

from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier

# ============================================================================
# 1. SETUP PATHS
# ============================================================================

DATA_DIR = os.path.join("..", "data", "plasticc")
FEATURES_PATH = os.path.join(DATA_DIR, "transient_features.csv")

RESULTS_DIR = os.path.join("..", "results")
os.makedirs(RESULTS_DIR, exist_ok=True)

print("=" * 70)
print("QUANTUM ML - OUTLIER-ROBUST VERSION")
print("=" * 70)
print(f"\nFeatures path: {FEATURES_PATH}")
print(f"Results dir: {RESULTS_DIR}\n")

# ============================================================================
# 2. LOAD DATA & ANALYZE FEATURES
# ============================================================================

features_df = pd.read_csv(FEATURES_PATH)
print(f"Loaded {len(features_df)} samples")
print(f"Available columns: {features_df.columns.tolist()}\n")

# Analyze feature statistics to find stable features
print("=" * 70)
print("FEATURE ANALYSIS BY CLASS")
print("=" * 70)

numeric_cols = features_df.select_dtypes(include=[np.number]).columns.tolist()
numeric_cols.remove('transient_id')  # Remove ID column

print("\nFeature means by class:")
feature_stats = features_df.groupby('label')[numeric_cols].mean()
print(feature_stats)

print("\nFeature standard deviations by class:")
feature_stds = features_df.groupby('label')[numeric_cols].std()
print(feature_stds)

# Calculate coefficient of variation (std/mean) to find stable features
print("\nCoefficient of Variation (lower is more stable):")
cv_by_class = feature_stds / (feature_stats.abs() + 1e-10)
print(cv_by_class)

# ============================================================================
# FEATURE SELECTION - ROBUST FEATURES ONLY
# ============================================================================

print("\n" + "=" * 70)
print("SELECTING ROBUST FEATURES")
print("=" * 70)

# Strategy: Use features with:
# 1. Lower coefficient of variation (more stable)
# 2. Good class separation
# 3. No extreme outliers

# OPTION 1: Stable features (try this first)
selected_features = ["flux_std", "flux_mean", "time_span"]

# OPTION 2: If flux features still have issues, try these:
# selected_features = ["n_points", "time_span", "flux_mean"]

# OPTION 3: Try delta features:
# selected_features = ["delta_flux", "flux_mean", "time_span"]

print(f"\nSelected features: {selected_features}")

# Verify features exist
for f in selected_features:
    if f not in features_df.columns:
        raise ValueError(f"Feature '{f}' not found! Available: {features_df.columns.tolist()}")

# ============================================================================
# 3. PREPARE DATA WITH OUTLIER HANDLING
# ============================================================================

X_raw = features_df[selected_features].values
y_str = features_df["label"].values

# Map labels
label_mapping = {"SNII": 0, "SNIa": 1}
y = np.array([label_mapping[val] for val in y_str])

print(f"\nRaw data shape: {X_raw.shape}")
print(f"Class distribution: {dict(zip(*np.unique(y, return_counts=True)))}")

# Check for NaN/inf
if np.any(np.isnan(X_raw)) or np.any(np.isinf(X_raw)):
    print("\nWARNING: NaN or Inf values detected!")
    X_raw = np.nan_to_num(X_raw, nan=0.0, posinf=1e10, neginf=-1e10)

# Analyze feature distributions BEFORE scaling
print("\n" + "=" * 70)
print("RAW FEATURE STATISTICS (BEFORE SCALING)")
print("=" * 70)
for i, feat in enumerate(selected_features):
    feat_data = X_raw[:, i]
    print(f"\n{feat}:")
    print(f"  Min:    {feat_data.min():.4f}")
    print(f"  Max:    {feat_data.max():.4f}")
    print(f"  Mean:   {feat_data.mean():.4f}")
    print(f"  Median: {np.median(feat_data):.4f}")
    print(f"  Std:    {feat_data.std():.4f}")
    print(f"  25%:    {np.percentile(feat_data, 25):.4f}")
    print(f"  75%:    {np.percentile(feat_data, 75):.4f}")
    print(f"  99%:    {np.percentile(feat_data, 99):.4f}")

# ============================================================================
# KEY FIX: OUTLIER-ROBUST SCALING
# ============================================================================

print("\n" + "=" * 70)
print("APPLYING OUTLIER-ROBUST SCALING")
print("=" * 70)

# Method 1: Clip outliers at 99th percentile
X_clipped = X_raw.copy()
for i in range(X_clipped.shape[1]):
    p99 = np.percentile(X_clipped[:, i], 99)
    p1 = np.percentile(X_clipped[:, i], 1)
    X_clipped[:, i] = np.clip(X_clipped[:, i], p1, p99)
    print(f"Feature {i} ({selected_features[i]}) clipped to [{p1:.4f}, {p99:.4f}]")

# Method 2: Apply log transform to reduce dynamic range (optional)
# Use this if features are still heavily skewed
X_processed = X_clipped.copy()

# Check if any features have large dynamic range (max/min > 1000)
for i in range(X_processed.shape[1]):
    feat_min = X_processed[:, i].min()
    feat_max = X_processed[:, i].max()
    dynamic_range = (feat_max - feat_min) / (abs(feat_min) + 1e-10)
    
    if dynamic_range > 100:
        print(f"  Feature {i} has large dynamic range ({dynamic_range:.1f}), applying log transform")
        # Shift to positive + log transform
        X_processed[:, i] = np.log1p(X_processed[:, i] - X_processed[:, i].min() + 1)

print()

# ============================================================================
# 4. TRAIN-TEST SPLIT
# ============================================================================

X_train, X_test, y_train, y_test = train_test_split(
    X_processed, y,
    test_size=0.2,
    random_state=42,
    stratify=y,
)

print(f"Train: {X_train.shape}, Test: {X_test.shape}")
print(f"Train class distribution: {dict(zip(*np.unique(y_train, return_counts=True)))}")
print(f"Test class distribution: {dict(zip(*np.unique(y_test, return_counts=True)))}\n")

# ============================================================================
# 5. SCALE TO QUANTUM ANGLES [0, π]
# ============================================================================

print("=" * 70)
print("SCALING TO QUANTUM ANGLES")
print("=" * 70)

# Use MinMaxScaler AFTER outlier removal
scaler = MinMaxScaler(feature_range=(0, np.pi))
X_train_angles = scaler.fit_transform(X_train)
X_test_angles = scaler.transform(X_test)

print("\nScaled feature ranges (train):")
print(f"  Min:    {X_train_angles.min(axis=0)}")
print(f"  Max:    {X_train_angles.max(axis=0)}")
print(f"  Mean:   {X_train_angles.mean(axis=0)}")
print(f"  Median: {np.median(X_train_angles, axis=0)}")

# CRITICAL CHECK: Are features well-distributed?
print("\nFeature distribution check:")
for i, feat in enumerate(selected_features):
    mean_val = X_train_angles[:, i].mean()
    median_val = np.median(X_train_angles[:, i])
    print(f"  {feat}: mean={mean_val:.3f}, median={median_val:.3f}")
    
    if mean_val < 0.3 or mean_val > 2.8:
        print(f"    ⚠️  WARNING: Feature concentrated near boundary!")
    elif abs(mean_val - np.pi/2) < 0.5:
        print(f"    ✓ Good: Feature centered around π/2")

print()

# ============================================================================
# 6. BUILD QUANTUM CIRCUIT
# ============================================================================

num_features = X_train_angles.shape[1]
num_qubits = num_features

print("=" * 70)
print(f"BUILDING {num_qubits}-QUBIT QUANTUM CIRCUIT")
print("=" * 70)

# Feature map with strong entanglement
feature_map = ZZFeatureMap(
    feature_dimension=num_features,
    reps=2,
    entanglement="full"
)

# Deeper ansatz for better expressivity
ansatz = TwoLocal(
    num_qubits=num_qubits,
    reps=3,
    rotation_blocks=["ry", "rz"],
    entanglement_blocks="cz",
    entanglement="full"
)

# Combine
from qiskit import QuantumCircuit

circuit = QuantumCircuit(num_qubits)
circuit.compose(feature_map, inplace=True)
circuit.compose(ansatz, inplace=True)

print(f"\nCircuit statistics:")
print(f"  Depth: {circuit.depth()}")
print(f"  Total parameters: {circuit.num_parameters}")
print(f"  Input params: {len(list(feature_map.parameters))}")
print(f"  Trainable params: {len(list(ansatz.parameters))}\n")

# ============================================================================
# 7. CREATE QUANTUM NEURAL NETWORK
# ============================================================================

estimator = Estimator()

input_params = list(feature_map.parameters)
weight_params = list(ansatz.parameters)

qnn = EstimatorQNN(
    estimator=estimator,
    circuit=circuit,
    input_params=input_params,
    weight_params=weight_params,
)

print("EstimatorQNN created")
print(f"  Input dimension: {len(input_params)}")
print(f"  Trainable weights: {len(weight_params)}\n")

# ============================================================================
# 8. CREATE QUANTUM CLASSIFIER WITH MORE ITERATIONS
# ============================================================================

# Try higher iterations since we fixed the scaling
optimizer = COBYLA(maxiter=300)  # Increased to 300

q_clf = NeuralNetworkClassifier(
    neural_network=qnn,
    optimizer=optimizer,
    one_hot=False,
)

print(f"Quantum classifier: COBYLA(maxiter=300)\n")

# ============================================================================
# 9. TRAIN QUANTUM MODEL
# ============================================================================

print("=" * 70)
print("TRAINING QUANTUM MODEL (this may take 5-7 minutes)")
print("=" * 70)

import time
start_time = time.time()

q_clf.fit(X_train_angles, y_train)

training_time = time.time() - start_time
print(f"\n✓ Training completed in {training_time:.1f}s ({training_time/60:.1f} min)")

# ============================================================================
# 10. EVALUATE
# ============================================================================

print("\n" + "=" * 70)
print("EVALUATION")
print("=" * 70)

# Predictions
y_pred_raw = q_clf.predict(X_test_angles)
print(f"\nRaw predictions: {np.unique(y_pred_raw)}")

y_pred_q = np.where(y_pred_raw <= 0, 0, 1)
print(f"Binary predictions: {np.unique(y_pred_q)}")
print(f"Prediction counts: {dict(zip(*np.unique(y_pred_q, return_counts=True)))}")

# Probability scores
y_proba_q = q_clf.predict_proba(X_test_angles)
scores = y_proba_q.ravel()
print(f"Score range: [{scores.min():.4f}, {scores.max():.4f}]")
print(f"Score mean: {scores.mean():.4f}, std: {scores.std():.4f}\n")

# Metrics
q_acc = accuracy_score(y_test, y_pred_q)
q_auc = roc_auc_score(y_test, scores)

print("=" * 70)
print("RESULTS")
print("=" * 70)
print(f"\n🎯 Accuracy: {q_acc:.1%} ({q_acc:.3f})")
print(f"📊 AUC:      {q_auc:.1%} ({q_auc:.3f})")

# Comparison
random_acc = 0.5
print(f"\n📈 Random baseline: {random_acc:.1%}")
print(f"📈 Improvement: {(q_acc - random_acc):.1%} ({q_acc - random_acc:+.3f})")

classical_acc = 0.75  # From your classical work
print(f"\n📊 Classical baseline: {classical_acc:.1%}")
print(f"📊 Gap to classical: {(q_acc - classical_acc):.1%} ({q_acc - classical_acc:+.3f})")

# Classification report
print("\n" + "=" * 70)
print("CLASSIFICATION REPORT")
print("=" * 70)
print(classification_report(
    y_test, y_pred_q,
    labels=[0, 1],
    target_names=["SNII (0)", "SNIa (1)"],
    zero_division=0,
))

# Confusion matrix
cm_q = confusion_matrix(y_test, y_pred_q, labels=[0, 1])
print("Confusion Matrix:")
print("                Predicted")
print("              SNII  SNIa")
print(f"Actual SNII   {cm_q[0,0]:3d}   {cm_q[0,1]:3d}")
print(f"       SNIa   {cm_q[1,0]:3d}   {cm_q[1,1]:3d}\n")

# Calculate balanced metrics
tn, fp, fn, tp = cm_q.ravel()
sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
balanced_acc = (sensitivity + specificity) / 2

print(f"Sensitivity (recall for SNIa): {sensitivity:.1%}")
print(f"Specificity (recall for SNII): {specificity:.1%}")
print(f"Balanced accuracy: {balanced_acc:.1%}\n")

# ============================================================================
# 11. SAVE RESULTS
# ============================================================================

results = {
    "model": "EstimatorQNN",
    "version": "outlier_robust",
    "features": selected_features,
    "preprocessing": {
        "outlier_clipping": "99th percentile",
        "scaling": "MinMaxScaler [0, π]",
        "log_transform": "conditional on dynamic range"
    },
    "num_qubits": num_qubits,
    "circuit_depth": int(circuit.depth()),
    "num_parameters": int(circuit.num_parameters),
    "trainable_params": len(weight_params),
    "training_time_seconds": float(training_time),
    "optimizer": "COBYLA",
    "max_iterations": 300,
    "feature_map": {
        "type": "ZZFeatureMap",
        "reps": 2,
        "entanglement": "full"
    },
    "ansatz": {
        "type": "TwoLocal",
        "reps": 3,
        "rotation_blocks": ["ry", "rz"],
        "entanglement_blocks": "cz",
        "entanglement": "full"
    },
    "results": {
        "test_samples": int(len(y_test)),
        "accuracy": float(q_acc),
        "auc": float(q_auc),
        "balanced_accuracy": float(balanced_acc),
        "sensitivity": float(sensitivity),
        "specificity": float(specificity),
        "confusion_matrix": cm_q.tolist(),
        "random_baseline": 0.5,
        "classical_baseline": 0.75,
    }
}

results_path = os.path.join(RESULTS_DIR, "plasticc_quantum_results_robust.json")
with open(results_path, 'w') as f:
    json.dump(results, f, indent=2)

print(f"Results saved to: {results_path}")

# ============================================================================
# 12. FINAL ASSESSMENT
# ============================================================================

print("\n" + "=" * 70)
print("FINAL ASSESSMENT")
print("=" * 70)

if q_acc < 0.52:
    print("\n⚠️  Model performed at random level (<52%)")
    print("\nPossible issues:")
    print("  1. Features still have poor separation")
    print("  2. Circuit might need different architecture")
    print("  3. Dataset too small for quantum advantage")
    print("  4. Try different feature combinations")
    
elif q_acc < 0.60:
    print("\n✓ Model learned something (52-60%)")
    print("\nThis is acceptable for quantum on small dataset.")
    print("Gap to classical (75%) expected due to:")
    print("  - Only 3 features vs 16 in classical")
    print("  - Small training set (480 samples)")
    print("  - NISQ hardware limitations")
    
elif q_acc < 0.70:
    print("\n✅ Good quantum performance! (60-70%)")
    print("\nThis is strong result for quantum ML on small dataset.")
    print("Demonstrates quantum circuits can learn meaningful patterns.")
    
else:
    print("\n🎉 Excellent quantum performance! (>70%)")
    print("\nQuantum approaching classical performance!")
    print("This is impressive for only 3 features.")

print("\n" + "=" * 70)
print("COMPLETE!")
print("=" * 70)

print("\nNext steps:")
print("  1. Document results in README")
print("  2. Compare quantum vs classical approaches")
print("  3. Discuss learnings about quantum ML limitations")
print("  4. Consider trying different feature sets if time permits")

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.
QUANTUM ML - OUTLIER-ROBUST VERSION

Features path: ../data/plasticc/transient_features.csv
Results dir: ../results

Loaded 600 samples
Available columns: ['transient_id', 'n_points', 'flux_min', 'flux_max', 'delta_flux', 'flux_mean', 'flux_std', 'time_span', 'max_slope', 'min_slope', 'mean_slope', 'label']

FEATURE ANALYSIS BY CLASS

Feature means by class:
         n_points   flux_min    flux_max  delta_flux  flux_mean   flux_std  \
label                                                                        
SNII   178.800000 -53.312604  451.000899  504.313503  34.258654  93.670573   
SNIa   203.193333 -50.560370  336.772262  387.332632  17.672213  57.700657   

        time_span    max_slope    min_slope  mean_slope  
label                                                    
SNII   944.824472  7891.975583 -6892.593720   34.580906 

/var/folders/4j/986s_f794zq87n31mh_7ls200000gn/T/ipykernel_80587/805784770.py:262: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()
/var/folders/4j/986s_f794zq87n31mh_7ls200000gn/T/ipykernel_80587/805784770.py:267: DeprecationWarning: V1 Primitives are deprecated as of qiskit-machine-learning 0.8.0 and will be removed no sooner than 4 months after the release date. Use V2 primitives for continued compatibility and support.
  qnn = EstimatorQNN(



✓ Training completed in 375.8s (6.3 min)

EVALUATION

Raw predictions: [-1.  1.]
Binary predictions: [0 1]
Prediction counts: {np.int64(0): np.int64(11), np.int64(1): np.int64(109)}
Score range: [-0.5372, 0.8517]
Score mean: 0.3503, std: 0.2717

RESULTS

🎯 Accuracy: 50.8% (0.508)
📊 AUC:      44.3% (0.443)

📈 Random baseline: 50.0%
📈 Improvement: 0.8% (+0.008)

📊 Classical baseline: 75.0%
📊 Gap to classical: -24.2% (-0.242)

CLASSIFICATION REPORT
              precision    recall  f1-score   support

    SNII (0)       0.55      0.10      0.17        60
    SNIa (1)       0.50      0.92      0.65        60

    accuracy                           0.51       120
   macro avg       0.53      0.51      0.41       120
weighted avg       0.53      0.51      0.41       120

Confusion Matrix:
                Predicted
              SNII  SNIa
Actual SNII     6    54
       SNIa     5    55

Sensitivity (recall for SNIa): 91.7%
Specificity (recall for SNII): 10.0%
Balanced accuracy: 50.8%

Resu

In [24]:
# ============================================================================
# 03_quantum_classifier.ipynb - FINAL VERSION WITH AUTO FEATURE SELECTION
# Quantum ML for PLAsTiCC Transient Classification
# ============================================================================

import os
import json
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import (
    accuracy_score, 
    classification_report, 
    confusion_matrix, 
    roc_auc_score
)
from scipy.stats import pointbiserialr

# Quantum imports
from qiskit.circuit.library import ZZFeatureMap, TwoLocal
from qiskit.primitives import Estimator
from qiskit_algorithms.optimizers import COBYLA

from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier

# ============================================================================
# 1. SETUP
# ============================================================================

DATA_DIR = os.path.join("..", "data", "plasticc")
FEATURES_PATH = os.path.join(DATA_DIR, "transient_features.csv")

RESULTS_DIR = os.path.join("..", "results")
os.makedirs(RESULTS_DIR, exist_ok=True)

print("=" * 70)
print("QUANTUM ML - FINAL VERSION (1072 SAMPLES)")
print("=" * 70)
print(f"\nFeatures path: {FEATURES_PATH}")
print(f"Results dir: {RESULTS_DIR}\n")

# ============================================================================
# 2. LOAD DATA & AUTO-SELECT TOP 3 FEATURES
# ============================================================================

features_df = pd.read_csv(FEATURES_PATH)
print(f"Loaded {len(features_df)} samples")
print(f"Class distribution:\n{features_df['label'].value_counts()}\n")

# All feature columns (16 features)
feature_cols = [
    "mag_min", "mag_max", "mag_mean", "mag_std", "mag_range",
    "flux_max", "flux_mean", "flux_std",
    "time_span", "rise_time", "decline_time", "rise_decline_ratio",
    "mean_rise_slope", "mean_decline_slope", "max_slope",
    "n_points"
]

X_full = features_df[feature_cols].values
y_str = features_df['label'].values

label_map = {'SNII': 0, 'SNIa': 1}
y = np.array([label_map[label] for label in y_str])

# ============================================================================
# FEATURE CORRELATION - AUTO-SELECT TOP 3
# ============================================================================

print("=" * 70)
print("AUTO-SELECTING TOP 3 FEATURES BY CORRELATION")
print("=" * 70)

correlations = []
for i, feat in enumerate(feature_cols):
    corr, pval = pointbiserialr(y, X_full[:, i])
    correlations.append({
        'feature': feat,
        'correlation': abs(corr),
        'correlation_signed': corr,
        'p_value': pval
    })

corr_df = pd.DataFrame(correlations).sort_values('correlation', ascending=False)

print("\nTop 10 features:")
print(corr_df.head(10).to_string(index=False))

# Select TOP 3
selected_features = corr_df.head(3)['feature'].tolist()

print(f"\n{'='*70}")
print(f"🎯 SELECTED FOR QUANTUM: {selected_features}")
print(f"{'='*70}")

# Show separation
print("\nClass separation analysis:")
for feat in selected_features:
    feat_idx = feature_cols.index(feat)
    feat_data = X_full[:, feat_idx]
    
    snia_vals = feat_data[y == 1]
    snii_vals = feat_data[y == 0]
    
    sep = abs(snia_vals.mean() - snii_vals.mean())
    sep_sigma = sep / snii_vals.std() if snii_vals.std() > 0 else 0
    
    print(f"\n{feat}:")
    print(f"  SNIa: μ={snia_vals.mean():.3f}, σ={snia_vals.std():.3f}")
    print(f"  SNII: μ={snii_vals.mean():.3f}, σ={snii_vals.std():.3f}")
    print(f"  Separation: {sep:.3f} ({sep_sigma:.2f}σ)")

# ============================================================================
# 3. PREPARE DATA WITH SELECTED FEATURES
# ============================================================================

X_selected = features_df[selected_features].values

print(f"\n{'='*70}")
print(f"DATA PREPARATION")
print(f"{'='*70}")
print(f"\nSelected features shape: {X_selected.shape}")
print(f"Labels shape: {y.shape}")

# Check for NaN/inf
if np.any(np.isnan(X_selected)) or np.any(np.isinf(X_selected)):
    print("⚠️  WARNING: NaN/Inf detected, cleaning...")
    X_selected = np.nan_to_num(X_selected, nan=0.0, posinf=1e10, neginf=-1e10)

# ============================================================================
# 4. OUTLIER-ROBUST PREPROCESSING
# ============================================================================

print(f"\n{'='*70}")
print("OUTLIER-ROBUST PREPROCESSING")
print(f"{'='*70}")

# Clip outliers at 99th percentile
X_clipped = X_selected.copy()
for i in range(X_clipped.shape[1]):
    p99 = np.percentile(X_clipped[:, i], 99)
    p1 = np.percentile(X_clipped[:, i], 1)
    X_clipped[:, i] = np.clip(X_clipped[:, i], p1, p99)
    print(f"Feature {i} ({selected_features[i]}): clipped to [{p1:.4f}, {p99:.4f}]")

# Apply log transform if high dynamic range
X_processed = X_clipped.copy()
for i in range(X_processed.shape[1]):
    feat_min = X_processed[:, i].min()
    feat_max = X_processed[:, i].max()
    dynamic_range = (feat_max - feat_min) / (abs(feat_min) + 1e-10)
    
    if dynamic_range > 100:
        print(f"  Feature {i}: high dynamic range ({dynamic_range:.1f}), applying log transform")
        X_processed[:, i] = np.log1p(X_processed[:, i] - X_processed[:, i].min() + 1)

# ============================================================================
# 5. TRAIN-TEST SPLIT
# ============================================================================

X_train, X_test, y_train, y_test = train_test_split(
    X_processed, y,
    test_size=0.2,
    random_state=42,
    stratify=y,
)

print(f"\n{'='*70}")
print(f"TRAIN-TEST SPLIT")
print(f"{'='*70}")
print(f"Train: {X_train.shape}, Test: {X_test.shape}")
print(f"Train distribution: {dict(zip(*np.unique(y_train, return_counts=True)))}")
print(f"Test distribution: {dict(zip(*np.unique(y_test, return_counts=True)))}")

# ============================================================================
# 6. SCALE TO QUANTUM ANGLES [0, π]
# ============================================================================

print(f"\n{'='*70}")
print("SCALING TO QUANTUM ANGLES")
print(f"{'='*70}")

scaler = MinMaxScaler(feature_range=(0, np.pi))
X_train_angles = scaler.fit_transform(X_train)
X_test_angles = scaler.transform(X_test)

print(f"\nAngle ranges (train):")
print(f"  Min:    {X_train_angles.min(axis=0)}")
print(f"  Max:    {X_train_angles.max(axis=0)}")
print(f"  Mean:   {X_train_angles.mean(axis=0)}")
print(f"  Median: {np.median(X_train_angles, axis=0)}")

# Check distribution quality
print("\nDistribution quality check:")
for i, feat in enumerate(selected_features):
    mean_val = X_train_angles[:, i].mean()
    median_val = np.median(X_train_angles[:, i])
    print(f"  {feat}: mean={mean_val:.3f}, median={median_val:.3f}", end="")
    
    if mean_val < 0.3 or mean_val > 2.8:
        print(" ⚠️  Near boundary")
    elif abs(mean_val - np.pi/2) < 0.5:
        print(" ✓ Well-centered")
    else:
        print("")

# ============================================================================
# 7. BUILD QUANTUM CIRCUIT
# ============================================================================

num_qubits = X_train_angles.shape[1]

print(f"\n{'='*70}")
print(f"BUILDING {num_qubits}-QUBIT QUANTUM CIRCUIT")
print(f"{'='*70}")

# Feature map
feature_map = ZZFeatureMap(
    feature_dimension=num_qubits,
    reps=2,
    entanglement="full"
)

# Ansatz
ansatz = TwoLocal(
    num_qubits=num_qubits,
    reps=3,
    rotation_blocks=["ry", "rz"],
    entanglement_blocks="cz",
    entanglement="full"
)

# Combined circuit
from qiskit import QuantumCircuit

circuit = QuantumCircuit(num_qubits)
circuit.compose(feature_map, inplace=True)
circuit.compose(ansatz, inplace=True)

print(f"\nCircuit statistics:")
print(f"  Depth: {circuit.depth()}")
print(f"  Parameters: {circuit.num_parameters}")
print(f"  Input params: {len(list(feature_map.parameters))}")
print(f"  Trainable params: {len(list(ansatz.parameters))}")

# ============================================================================
# 8. CREATE QNN
# ============================================================================

estimator = Estimator()

input_params = list(feature_map.parameters)
weight_params = list(ansatz.parameters)

qnn = EstimatorQNN(
    estimator=estimator,
    circuit=circuit,
    input_params=input_params,
    weight_params=weight_params,
)

print(f"\nEstimatorQNN created:")
print(f"  Input dimension: {len(input_params)}")
print(f"  Trainable weights: {len(weight_params)}")

# ============================================================================
# 9. CREATE CLASSIFIER
# ============================================================================

optimizer = COBYLA(maxiter=300)

q_clf = NeuralNetworkClassifier(
    neural_network=qnn,
    optimizer=optimizer,
    one_hot=False,
)

print(f"\nQuantum classifier: COBYLA(maxiter=300)")

# ============================================================================
# 10. TRAIN
# ============================================================================

print(f"\n{'='*70}")
print("TRAINING QUANTUM MODEL")
print(f"{'='*70}")
print("This will take 5-10 minutes...\n")

import time
start_time = time.time()

q_clf.fit(X_train_angles, y_train)

training_time = time.time() - start_time
print(f"\n✓ Training completed in {training_time:.1f}s ({training_time/60:.1f} min)")

# ============================================================================
# 11. EVALUATE
# ============================================================================

print(f"\n{'='*70}")
print("EVALUATION")
print(f"{'='*70}")

# Predictions
y_pred_raw = q_clf.predict(X_test_angles)
y_pred_q = np.where(y_pred_raw <= 0, 0, 1)

# Probability scores
y_proba_q = q_clf.predict_proba(X_test_angles)
scores = y_proba_q.ravel()

# Metrics
q_acc = accuracy_score(y_test, y_pred_q)
q_auc = roc_auc_score(y_test, scores)

print(f"\n{'='*70}")
print("QUANTUM RESULTS")
print(f"{'='*70}")
print(f"\n🎯 Accuracy: {q_acc:.1%} ({q_acc:.3f})")
print(f"📊 AUC:      {q_auc:.1%} ({q_auc:.3f})")

# Comparison
print(f"\n📈 Random baseline:    50.0%")
print(f"📈 Quantum vs random:  {(q_acc - 0.5):.1%} ({q_acc - 0.5:+.3f})")
print(f"\n📊 Classical (Ensemble): 74.4%")
print(f"📊 Gap to classical:     {(q_acc - 0.744):.1%} ({q_acc - 0.744:+.3f})")

# Classification report
print(f"\n{'='*70}")
print("CLASSIFICATION REPORT")
print(f"{'='*70}")
print(classification_report(
    y_test, y_pred_q,
    labels=[0, 1],
    target_names=["SNII", "SNIa"],
    zero_division=0,
))

# Confusion matrix
cm_q = confusion_matrix(y_test, y_pred_q, labels=[0, 1])
print("Confusion Matrix:")
print("              SNII  SNIa")
print(f"Actual SNII   {cm_q[0,0]:3d}   {cm_q[0,1]:3d}")
print(f"       SNIa   {cm_q[1,0]:3d}   {cm_q[1,1]:3d}")

# Balanced metrics
tn, fp, fn, tp = cm_q.ravel()
sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
balanced_acc = (sensitivity + specificity) / 2

print(f"\nSensitivity (SNIa recall): {sensitivity:.1%}")
print(f"Specificity (SNII recall): {specificity:.1%}")
print(f"Balanced accuracy:         {balanced_acc:.1%}")

# ============================================================================
# 12. SAVE RESULTS
# ============================================================================

results = {
    "dataset_size": int(len(features_df)),
    "train_size": int(len(X_train)),
    "test_size": int(len(X_test)),
    "model": "EstimatorQNN",
    "features": selected_features,
    "num_qubits": num_qubits,
    "circuit_depth": int(circuit.depth()),
    "trainable_params": len(weight_params),
    "training_time_seconds": float(training_time),
    "optimizer": "COBYLA",
    "max_iterations": 300,
    "results": {
        "accuracy": float(q_acc),
        "auc": float(q_auc),
        "balanced_accuracy": float(balanced_acc),
        "sensitivity": float(sensitivity),
        "specificity": float(specificity),
        "confusion_matrix": cm_q.tolist(),
    },
    "baselines": {
        "random": 0.5,
        "classical_ensemble": 0.744,
        "classical_random_forest": 0.758,
    }
}

results_path = os.path.join(RESULTS_DIR, "plasticc_quantum_results_final.json")
with open(results_path, 'w') as f:
    json.dump(results, f, indent=2)

print(f"\n✓ Results saved: {results_path}")

# ============================================================================
# 13. FINAL ASSESSMENT
# ============================================================================

print(f"\n{'='*70}")
print("FINAL ASSESSMENT")
print(f"{'='*70}")

if q_acc < 0.52:
    status = "⚠️  Random level"
    msg = "Model did not learn meaningful patterns"
elif q_acc < 0.60:
    status = "✓ Minimal learning"
    msg = "Model learned something, but gap to classical remains large"
elif q_acc < 0.70:
    status = "✅ Good performance"
    msg = "Strong quantum learning! Approaching classical performance"
else:
    status = "🎉 Excellent!"
    msg = "Quantum rivaling classical performance!"

print(f"\n{status}: {q_acc:.1%} accuracy")
print(f"{msg}")

print(f"\n{'='*70}")
print("PROJECT COMPLETE!")
print(f"{'='*70}")

QUANTUM ML - FINAL VERSION (1072 SAMPLES)

Features path: ../data/plasticc/transient_features.csv
Results dir: ../results

Loaded 1072 samples
Class distribution:
label
SNII    549
SNIa    523
Name: count, dtype: int64

AUTO-SELECTING TOP 3 FEATURES BY CORRELATION

Top 10 features:
           feature  correlation  correlation_signed      p_value
         time_span     0.280059           -0.280059 9.061588e-21
      decline_time     0.268915           -0.268915 3.248802e-19
           mag_max     0.150529            0.150529 7.382729e-07
           mag_std     0.142485            0.142485 2.818433e-06
          mag_mean     0.133558            0.133558 1.146827e-05
         mag_range     0.104832            0.104832 5.864396e-04
         rise_time     0.096462           -0.096462 1.567008e-03
mean_decline_slope     0.063565            0.063565 3.744478e-02
         flux_mean     0.041182           -0.041182 1.778611e-01
           mag_min     0.033363            0.033363 2.750981e-01

🎯

/var/folders/4j/986s_f794zq87n31mh_7ls200000gn/T/ipykernel_80587/2605216957.py:252: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()
/var/folders/4j/986s_f794zq87n31mh_7ls200000gn/T/ipykernel_80587/2605216957.py:257: DeprecationWarning: V1 Primitives are deprecated as of qiskit-machine-learning 0.8.0 and will be removed no sooner than 4 months after the release date. Use V2 primitives for continued compatibility and support.
  qnn = EstimatorQNN(



✓ Training completed in 600.6s (10.0 min)

EVALUATION

QUANTUM RESULTS

🎯 Accuracy: 52.1% (0.521)
📊 AUC:      58.7% (0.587)

📈 Random baseline:    50.0%
📈 Quantum vs random:  2.1% (+0.021)

📊 Classical (Ensemble): 74.4%
📊 Gap to classical:     -22.3% (-0.223)

CLASSIFICATION REPORT
              precision    recall  f1-score   support

        SNII       0.59      0.20      0.30       110
        SNIa       0.51      0.86      0.64       105

    accuracy                           0.52       215
   macro avg       0.55      0.53      0.47       215
weighted avg       0.55      0.52      0.46       215

Confusion Matrix:
              SNII  SNIa
Actual SNII    22    88
       SNIa    15    90

Sensitivity (SNIa recall): 85.7%
Specificity (SNII recall): 20.0%
Balanced accuracy:         52.9%

✓ Results saved: ../results/plasticc_quantum_results_final.json

FINAL ASSESSMENT

✓ Minimal learning: 52.1% accuracy
Model learned something, but gap to classical remains large

PROJECT COMPLETE!
